In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import geopandas as gpd
from shapely.geometry import Point

# 나눔고딕 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_prop = fm.FontProperties(fname=font_path, size=12)
plt.rc('font', family='NanumGothic')

# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# CCTV 위치 데이터

In [ ]:
columns = ['1','2','3','4','5','6','7','8','9','10','11','12']
df_cctv = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/소현_유동인구 상권 및 시설 위치/cctv 위치/서울 cctv.csv', names = columns)
df_cctv.head()

,1,2,3,4,5,6,7,8,9,10,11,12
0,1,서초구,양재2동 산4-14,산불감시,37.432124,127.047291,1,2021-01-19,서울서초경찰서,서울서초,G30000001329813,서울청
1,2,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434221,126.903997,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청
2,3,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434303,126.903990,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청
3,4,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434312,126.904080,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청
4,5,금천구,시흥대로2길 8-49 주변 [시흥3동 978-6],방범,37.434383,126.903963,1,2022-02-23,서울금천경찰서,서울금천,G30000001332675,서울청


In [ ]:
# 필요한 열만 남기기
df_cctv = df_cctv[['2','4','5','6']]
df_cctv.head()

,2,4,5,6
0,서초구,산불감시,37.432124,127.047291
1,금천구,방범,37.434221,126.903997
2,금천구,방범,37.434303,126.903990
3,금천구,방범,37.434312,126.904080
4,금천구,방범,37.434383,126.903963


In [ ]:
# CCTV 목적별 count
df_cctv['4'].value_counts()

4
방범          47270
어린이안전        4833
도심공원         4213
불법주정차        3393
쓰레기 무단투기      472
치수방재          415
시설물 관리        409
미세먼지          243
산불감시           62
고정식            58
회전식            46
자전거보관소         30
Name: count, dtype: int64

In [ ]:
# 방범, 어린이 안전, 불법 주정차 데이터만 남기기
df_cctv = df_cctv[df_cctv['4'].isin(['방범','어린이안전','불법주정차'])]
df_cctv['4'].value_counts()

4
방범       47270
어린이안전     4833
불법주정차     3393
Name: count, dtype: int64

In [ ]:
# 행정동 경계 데이터(위경도 변환용)
gdf_dong = gpd.read_file('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/소현_유동인구 상권 및 시설 위치/행정동경계.geojson')

In [ ]:
df_cctv['geometry'] = df_cctv.apply(lambda row: Point([row['6'], row['5']]), axis=1)
gdf_points = gpd.GeoDataFrame(df_cctv, geometry='geometry')

In [ ]:
gdf_points.crs = {'init': 'epsg:4326'}
gdf_dong = gdf_dong.to_crs(epsg=4326)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
gdf_result = gpd.sjoin(gdf_points, gdf_dong, how='left', op = 'within')
gdf_result.head()

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-18-e3ad58698bf9>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  gdf_result = gpd.sjoin(gdf_points, gdf_dong, how='left', op = 'within')


,2,4,5,6,geometry,index_right,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,temp,sggnm,adm_cd8
1,금천구,방범,37.434221,126.903997,POINT (126.90400 37.43422),1256,경기도 안양시만안구 석수1동,3104160,4117159000,41171,41,경기도,안양시만안구 석수1동,안양시만안구,31041600
2,금천구,방범,37.434303,126.903990,POINT (126.90399 37.43430),1256,경기도 안양시만안구 석수1동,3104160,4117159000,41171,41,경기도,안양시만안구 석수1동,안양시만안구,31041600
3,금천구,방범,37.434312,126.904080,POINT (126.90408 37.43431),1256,경기도 안양시만안구 석수1동,3104160,4117159000,41171,41,경기도,안양시만안구 석수1동,안양시만안구,31041600
4,금천구,방범,37.434383,126.903963,POINT (126.90396 37.43438),1256,경기도 안양시만안구 석수1동,3104160,4117159000,41171,41,경기도,안양시만안구 석수1동,안양시만안구,31041600
5,서초구,방범,37.434617,127.069873,POINT (127.06987 37.43462),356,서울특별시 서초구 내곡동,1122068,1165066000,11650,11,서울특별시,서초구 내곡동,서초구,11220680


In [ ]:
# df_cctv에 결합
df_cctv['행정동명'] = gdf_result['temp']
df_cctv['행정동 코드'] = gdf_result['adm_cd2']
df_cctv.head()

,2,4,5,6,geometry,행정동명,행정동 코드
1,금천구,방범,37.434221,126.903997,POINT (126.9039971 37.4342208),안양시만안구 석수1동,4117159000
2,금천구,방범,37.434303,126.903990,POINT (126.9039902 37.43430303),안양시만안구 석수1동,4117159000
3,금천구,방범,37.434312,126.904080,POINT (126.9040799 37.43431204),안양시만안구 석수1동,4117159000
4,금천구,방범,37.434383,126.903963,POINT (126.9039632 37.43438346),안양시만안구 석수1동,4117159000
5,서초구,방범,37.434617,127.069873,POINT (127.0698735 37.4346168),서초구 내곡동,1165066000


In [ ]:
# 경계에 걸쳐서 행정동이 서울을 벗어난 데이터
df_cctv[~df_cctv['행정동 코드'].str.startswith('11')]

,2,4,5,6,geometry,행정동명,행정동 코드
1,금천구,방범,37.434221,126.903997,POINT (126.9039971 37.4342208),안양시만안구 석수1동,4117159000
2,금천구,방범,37.434303,126.903990,POINT (126.9039902 37.43430303),안양시만안구 석수1동,4117159000
3,금천구,방범,37.434312,126.904080,POINT (126.9040799 37.43431204),안양시만안구 석수1동,4117159000
4,금천구,방범,37.434383,126.903963,POINT (126.9039632 37.43438346),안양시만안구 석수1동,4117159000
1517,강남구,방범,37.462200,127.107310,POINT (127.10731 37.4622),성남시수정구 신촌동,4113163000


In [ ]:
# 전처리 기준 "서울" 내 데이터만 추출
df_cctv = df_cctv[df_cctv['행정동 코드'].str.startswith('11')]
df_cctv.columns = ['구','목적','위도','경도','geometry','행정동명','행정동 코드']
df_cctv.head()

,구,목적,위도,경도,geometry,행정동명,행정동 코드
5,서초구,방범,37.434617,127.069873,POINT (127.0698735 37.4346168),서초구 내곡동,1165066000
6,서초구,방범,37.434618,127.069861,POINT (127.0698609 37.4346177),서초구 내곡동,1165066000
7,서초구,방범,37.434619,127.069887,POINT (127.0698867 37.43461925),서초구 내곡동,1165066000
8,금천구,방범,37.434622,126.905285,POINT (126.9052852 37.43462204),금천구 시흥3동,1154569000
9,서초구,방범,37.434627,127.069871,POINT (127.0698707 37.43462687),서초구 내곡동,1165066000


In [ ]:
# 행정동별 CCTV 개수 count
df_cctv_count = df_cctv.groupby(['행정동명']).count()
df_cctv_count.reset_index(inplace=True)
df_cctv_count = df_cctv_count[['행정동명','행정동 코드']]
df_cctv_count.rename(columns={'행정동 코드':'CCTV 개수'}, inplace=True)
df_cctv_count.head()

,행정동명,CCTV 개수
0,강남구 개포1동,57
1,강남구 개포2동,231
2,강남구 개포3동,108
3,강남구 개포4동,381
4,강남구 논현1동,478


In [ ]:
# 행정동 코드와 결합
def find_code(row, df_cctv):
    match = df_cctv[df_cctv['행정동명'] == row['행정동명']]
    if not match.empty:
        return match['행정동 코드'].values[0]
    else:
        return None

df_cctv_count['행정동 코드'] = df_cctv_count.apply(find_code, axis=1, args=(df_cctv,))

# 전처리 최종
df_cctv_count.head()

,행정동명,CCTV 개수,행정동 코드
0,강남구 개포1동,57,1168066000
1,강남구 개포2동,231,1168067000
2,강남구 개포3동,108,1168067500
3,강남구 개포4동,381,1168069000
4,강남구 논현1동,478,1168052100


In [ ]:
# 결측치 확인
df_cctv_count.isnull().sum()

행정동명       0
CCTV 개수    0
행정동 코드     0
dtype: int64

In [ ]:
df_cctv_count.to_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/소현_유동인구 상권 및 시설 위치/cctv 위치/행정동별 cctv 개수.csv')